In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"
path_pdb = f"{path_project}/sword2_DepoSeq_pt2"
path_db = "/home/conchae/depolymerase/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan_domains.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/seekfold_PPT_domains"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_domain = []
for file in os.listdir(f"{path_pdb}") :
    try : 
        for file_2 in os.listdir(f"{path_pdb}/{file}/{file}_A/Protein_Units") :
            if file_2[-3:] == "pdb" :
                path = f"{path_pdb}/{file}/{file}_A/Protein_Units/{file_2}"
                paths_domain.append(path)
    except Exception as e:
        print(file)    

if __name__ == '__main__':
    with ThreadPool(2) as p:
        p.map(seek_beta_helix, paths_domain)
    
# ******************************************************************************************
#!/bin/bash
#BATCH --job-name=seek_PPT__
#SBATCH --partition=medium 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=40
#SBATCH --mem=90gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=seek_PPT__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/script_files/seek_betahelix.ppt.py

In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/home/conchae/PhageDepo_pdb"
path_pdb = f"{path_project}/phagedepo_out"
path_db = "/home/conchae/depolymerase/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/seekfold_PhageDepo_full"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)
    outfile.write(f"{protein_id}\n")


pdb_repeat =   ['4117', '2634', '2765', '4265', '2796', '1429', '6440', '4248', '1436', '3085', '1805', '1218', '817', '3947', '3054', '2229', '1424', '3074', '6514', '1728', '6458', '2106', '1840', '4510', '3210', '226', '3933', '6022', '2956', '6561', '4775', '218', '5078', '1013', '3651', '3767', '4110', '1693', '4042', '1574', '5059', '2963', '6627', '3813', '1111', '5343', '5007', '642', '5860', '3595', '3854', '15', '3351', '36', '2891', '4160', '3498', '5631', '2089', '2945', '3317', '1147', '3678', '6108', '4999', '141', '2464', '360', '1461', '4704', '1559', '1789', '5594', '3010', '1043', '6426', '502', '4552', '209', '2747', '6246', '319', '819', '6442', '1862', '5983', '1597']  
    
paths_pdb = []
for file in os.listdir(f"{path_pdb}") :
    path = f"{path_pdb}/{file}"
    paths_pdb.append(path)
    


if __name__ == '__main__':
    with ThreadPool(20) as p:
        p.map(seek_beta_helix, paths_domain)
    
# ******************************************************************************************
#!/bin/bash
#BATCH --job-name=sf_phagedepo__
#SBATCH --qos=short 
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=40
#SBATCH --mem=40gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=seek_phagedepo__%j.log 

source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate HH-suite3

python /home/conchae/PhageDepo_pdb/script_files/seek_full.phagedepo.py

***
# Scan the results

In [ ]:
import sys
import os 
import pandas as pd 
from Bio import SeqIO
from tqdm import tqdm
import json

path_project = "/home/conchae/PhageDepo_pdb"
dir_out_full = f"{path_project}/seekfold_PhageDepo_full"
dir_out = f"{path_project}/seekfold_PhageDepo_domains"

path_swords_out = f"{path_project}/rbp_work/sword2_depo"
path_info = f"/home/conchae/depolymerase/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob","alntmscore"]
dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

errors = {'A1l_00058_A_1_129_274.pdb',
 'AF-A0A089GJA0-F1-model_v4.pdb',
 'AF-A0A4V2M274-F1-model_v4.pdb',
 'AF-A0A7W9JCS6-F1-model_v4.pdb','AF-A0A658Y395-F1-model_v4.pdb'}

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")


dico_domain_result = {}
for protein in tqdm(dico_out) :
    the_winner = (0,0,0)
    try : 
        # Collecting data for each proteins
        results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}.out" , sep = "\t" , names = header_seekfold) 
        results_full_prot = results_full_prot[~results_full_prot["target"].isin(errors)]
        for _,row in results_full_prot.iterrows():
            fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
            if fold != "jelly-roll" :
                full_score = int(row["bits"] )
                target = row["target"]
                break
        the_full_prot = (full_score , f"Full protein {protein}")
        for path in dico_out[protein] :
            if os.path.getsize(path) > 0 :
                results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold)
                results_df = results_df[results_df["target"] == target]
                if results_df.empty == False :
                    aa_end = results_df["query"].values[0].split("_")[-1].split(".pdb")[0]
                    aa_start = results_df["query"].values[0].split("_")[-2]
                    PU_length = int(aa_end) - int(aa_start)
                    score = int(results_df["bits"].values[0])
                    contender = (score , PU_length ,results_df["query"].values[0])
                    if contender[0] > the_winner[0] :
                        the_winner = contender
                    elif contender[0] == the_winner[0] :
                        if contender[1] < the_winner[1] :
                            the_winner = contender
                    else :
                        pass
        if the_full_prot[0] > (the_winner[0] + 25) :
            the_winner = the_full_prot 
        if the_winner[0] > 20 :
            dico_domain_result[protein] = the_winner
    except Exception as e :
        print(protein , e)
    
with open(f"{path_project}/PhageDepo.domains.json", "w") as outfile:
    json.dump(dico_domain_result, outfile)

In [ ]:
#dico_domain_result = {}
full_prot_repeat = []
missing_fold = set()
for protein in tqdm(dico_out) :
    the_winner = (0,0,0)
    # Collecting data for each proteins
    if os.path.isfile (f"{dir_out_full}/{protein}.out") :
        results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}.out" , sep = "\t" , names = header_seekfold).fillna("nothing")
        results_full_prot = results_full_prot[~results_full_prot["target"].isin(errors)]
        print(results_full_prot)
        for _,row in results_full_prot.iterrows():
            try : 
                fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
            except IndexError :
                missing_fold.add(row["target"])
                print(row["target"])
    else :
        full_prot_repeat.append(protein)
        
        
        
            break
    the_full_prot = (full_score , f"Full protein {protein}")
    for path in dico_out[protein] :
        if os.path.getsize(path) > 0 :
            results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold)
            results_df = results_df[results_df["target"] == target]
            if results_df.empty == False :
                aa_end = results_df["query"].values[0].split("_")[-1].split(".pdb")[0]
                aa_start = results_df["query"].values[0].split("_")[-2]
                PU_length = int(aa_end) - int(aa_start)
                score = int(results_df["bits"].values[0])
                contender = (score , PU_length ,results_df["query"].values[0])
                if contender[0] > the_winner[0] :
                    the_winner = contender
                elif contender[0] == the_winner[0] :
                    if contender[1] < the_winner[1] :
                        the_winner = contender
                else :
                    pass
    if the_full_prot[0] > (the_winner[0] + 25) :
        the_winner = the_full_prot 
    if the_winner[0] > 20 :
        dico_domain_result[protein] = the_winner
